In [ ]:
!pip install datasets transformers torch pandas scikit-learn scipy matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

save_path = '/content/gdrive/My Drive/NLP Final Project/best_model.pth'
# Ensure directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, get_dataset_config_names
from scipy.stats import pearsonr, spearmanr

In [ ]:
# Step 1: Data Preparation
def load_semrel2024_data():
    dataset_url = "SemRel/SemRel2024"

    # Get the list of available languages (configs)
    languages = get_dataset_config_names(dataset_url)

    # Load datasets for each language
    datasets = {}
    languages_with_train = []
    for lang in languages:
        try:
            lang_dataset = load_dataset(dataset_url, lang)
            datasets[lang] = lang_dataset
            if 'train' in lang_dataset:
                languages_with_train.append(lang)
        except Exception as e:
            print(f"Error loading dataset for language {lang}: {e}")

    return datasets, languages, languages_with_train

datasets, all_languages, train_languages = load_semrel2024_data()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/48.0k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/375 [00:00<?, ? examples/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/67.4k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/34.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1146 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/297 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/130 [00:00<?, ? examples/s]

In [ ]:
# Step 2: Model Selection and Tokenization

model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
# Step 3: Dataset Class

class SemRelDataset(Dataset):
    def __init__(self, data, tokenizer, lang):
        self.data = data
        self.tokenizer = tokenizer
        self.lang = lang

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = self.tokenizer(item['sentence1'], item['sentence2'],
                                  padding="max_length", truncation=True, return_tensors="pt")
        return {
            'sentence1': item['sentence1'],
            'sentence2': item['sentence2'],
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(item['label'], dtype=torch.float),
            'language': self.lang
        }

In [ ]:
# Create datasets for each split and language
train_datasets = {lang: SemRelDataset(datasets[lang]['train'], tokenizer, lang) for lang in train_languages}
val_datasets = {lang: SemRelDataset(datasets[lang]['dev'], tokenizer, lang) for lang in all_languages if 'dev' in datasets[lang]}
test_datasets = {lang: SemRelDataset(datasets[lang]['test'], tokenizer, lang) for lang in all_languages if 'test' in datasets[lang]}

In [ ]:
# Create combined dataset with all available languages
combined_train_dataset = ConcatDataset(list(train_datasets.values()))
combined_val_dataset = ConcatDataset(list(val_datasets.values()))
combined_test_dataset = ConcatDataset(list(test_datasets.values()))

In [ ]:
# Create DataLoaders to allow us to load data in batches to the model
train_dataloader = DataLoader(combined_train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(combined_val_dataset, batch_size=16)
test_dataloader = DataLoader(combined_test_dataset, batch_size=16)

In [ ]:
print(f"\nCombined train dataset size: {len(combined_train_dataset)}")
print(f"Combined validation dataset size: {len(combined_val_dataset)}")
print(f"Combined test dataset size: {len(combined_test_dataset)}")


Combined train dataset size: 15054
Combined validation dataset size: 2471
Combined test dataset size: 8732


In [ ]:
# Step 4: Model Architecture

class SemRelModel(torch.nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model
        self.regressor = torch.nn.Linear(base_model.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        return self.regressor(pooled_output)

In [ ]:
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
import time
import logging
import wandb  # Optional: for experiment tracking

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def train(model, train_dataloader, val_dataloader, num_epochs, device,
          early_stopping_patience=3, use_wandb=False):
    """
    Training function with enhanced monitoring and early stopping

    Args:
        model: The PyTorch model to train
        train_dataloader: DataLoader for training data
        val_dataloader: DataLoader for validation data
        num_epochs: Number of epochs to train
        device: Device to train on ('cuda' or 'cpu')
        early_stopping_patience: Number of epochs to wait before early stopping
        use_wandb: Whether to log metrics to Weights & Biases
    """
    optimizer = AdamW(model.parameters(), lr=2e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=len(train_dataloader) * num_epochs
    )
    criterion = torch.nn.MSELoss()

    # Initialize tracking variables
    best_val_loss = float('inf')
    training_stats = []
    start_time = time.time()

    if use_wandb:
        wandb.init(project="regression_training")

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        epoch_start_time = time.time()

        # Progress bar for training
        train_progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]')

        for batch_idx, batch in enumerate(train_progress_bar):
            optimizer.zero_grad()

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs.squeeze(), labels)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()

            # Update metrics
            total_train_loss += loss.item()
            current_lr = scheduler.get_last_lr()[0]

            # Update progress bar
            train_progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'lr': f'{current_lr:.2e}'
            })

        avg_train_loss = total_train_loss / len(train_dataloader)

        # Validation phase
        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []

        # Progress bar for validation
        val_progress_bar = tqdm(val_dataloader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]')

        with torch.no_grad():
            for batch in val_progress_bar:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids, attention_mask)
                batch_loss = criterion(outputs.squeeze(), labels)
                val_loss += batch_loss.item()

                # Collect predictions and labels for additional metrics
                all_preds.extend(outputs.squeeze().cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                val_progress_bar.set_postfix({'loss': f'{batch_loss.item():.4f}'})

        # Calculate validation metrics
        val_loss /= len(val_dataloader)
        mae = mean_absolute_error(all_labels, all_preds)
        r2 = r2_score(all_labels, all_preds)

        # Calculate time statistics
        epoch_time = time.time() - epoch_start_time
        total_time = time.time() - start_time

        # Log metrics
        epoch_stats = {
            'epoch': epoch + 1,
            'train_loss': avg_train_loss,
            'val_loss': val_loss,
            'mae': mae,
            'r2': r2,
            'learning_rate': current_lr,
            'epoch_time': epoch_time,
            'total_time': total_time
        }
        training_stats.append(epoch_stats)

        if use_wandb:
            wandb.log(epoch_stats)

        # Print metrics
        logger.info(f"\nEpoch {epoch+1}/{num_epochs}")
        logger.info(f"Training Loss: {avg_train_loss:.4f}")
        logger.info(f"Validation Loss: {val_loss:.4f}")
        logger.info(f"MAE: {mae:.4f}")
        logger.info(f"R² Score: {r2:.4f}")
        logger.info(f"Learning Rate: {current_lr:.2e}")
        logger.info(f"Epoch Time: {epoch_time:.2f}s")
        logger.info(f"Total Time: {total_time:.2f}s")

        # Model checkpoint saving
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'val_loss': val_loss,
                'train_loss': avg_train_loss,
            }, save_path)
            logger.info("Saved new best model!")
            patience_counter = 0
        else:
            patience_counter += 1

        # Early stopping
        if patience_counter >= early_stopping_patience:
            logger.info(f"Early stopping triggered after {epoch + 1} epochs")
            break

    return training_stats

In [ ]:
# Step 6: Evaluation Function

def evaluate(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []
    languages = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask)
            predictions.extend(outputs.squeeze().cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            languages.extend(batch['language'])

    predictions = np.array(predictions)
    true_labels = np.array(true_labels)

    overall_mae = np.mean(np.abs(predictions - true_labels))
    overall_pearson, _ = pearsonr(predictions, true_labels)
    overall_spearman, _ = spearmanr(predictions, true_labels)

    #Language-specific evaluation
    lang_results = {}
    for lang in set(languages):
        lang_mask = np.array(languages) == lang
        lang_pred = predictions[lang_mask]
        lang_true = true_labels[lang_mask]

        lang_mae = np.mean(np.abs(lang_pred - lang_true))
        lang_pearson, _ = pearsonr(lang_pred, lang_true)
        lang_spearman, _ = spearmanr(lang_pred, lang_true)

        lang_results[lang] = {
            'mae': lang_mae,
            'pearson': lang_pearson,
            'spearman': lang_spearman
        }

    return overall_mae, overall_pearson, overall_spearman, lang_results

In [ ]:
# Step 7: Main Execution

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SemRelModel(base_model).to(device)

# # Train the model
# stats = train(
#     model=model,
#     train_dataloader=train_dataloader,
#     val_dataloader=val_dataloader,
#     num_epochs=8,
#     device=device,
#     use_wandb=False  # Set to True if using Weights & Biases
# )

In [ ]:
checkpoint = torch.load(save_path, weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/My Drive/NLP Final Project/best_model.pth'

In [ ]:
# Evaluate the model
overall_mae, overall_pearson, overall_spearman, lang_results = evaluate(model, test_dataloader, device)

print(f"Overall MAE: {overall_mae:.4f}")
print(f"Overall Pearson correlation: {overall_pearson:.4f}")
print(f"Overall Spearman correlation: {overall_spearman:.4f}")

print("\nLanguage-specific results:")
for lang, metrics in lang_results.items():
    print(f"\nLanguage: {lang}")
    print(f"MAE: {metrics['mae']:.4f}")
    print(f"Pearson correlation: {metrics['pearson']:.4f}")
    print(f"Spearman correlation: {metrics['spearman']:.4f}")

In [ ]:
# Step 8: Error Analysis

def analyze_errors(model, dataloader, device, languages):
    model.eval()
    errors = {lang: [] for lang, _ in languages}

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            batch_languages = batch['language']

            outputs = model(input_ids, attention_mask)
            predictions = outputs.squeeze().cpu().numpy()
            true_labels = labels.cpu().numpy()

            batch_errors = np.abs(predictions - true_labels)

            for error, lang in zip(batch_errors, batch_languages):
                errors[lang].append(error)

    for lang, _ in languages:
        lang_errors = np.array(errors[lang])
        print(f"\nLanguage: {lang}")
        print(f"Mean Error: {np.mean(lang_errors):.4f}")
        print(f"Median Error: {np.median(lang_errors):.4f}")
        print(f"90th Percentile Error: {np.percentile(lang_errors, 90):.4f}")

analyze_errors(model, test_dataloader, device, lang_results.items())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_metrics_by_language(lang_results):
    metrics = ['mae', 'pearson', 'spearman']
    languages = list(lang_results.keys())

    languages = [lang for lang in languages if lang != 'esp']

    # Generate a color palette with unique colors for each language
    palette = sns.color_palette("husl", len(languages))
    color_mapping = {lang: palette[i] for i, lang in enumerate(languages)}

    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    fig.suptitle('Metrics Comparison Across Languages', fontsize=16)

    for i, metric in enumerate(metrics):
        values = [lang_results[lang][metric] for lang in languages]
        sns.barplot(x=languages, y=values, ax=axes[i], palette=[color_mapping[lang] for lang in languages])
        axes[i].set_title(f'{metric.capitalize()} by Language')
        axes[i].set_xticks(range(len(languages)))
        axes[i].set_xticklabels(languages, rotation=45, ha='right')
        axes[i].set_ylabel(metric.upper())

    plt.tight_layout()
    plt.savefig('metrics_by_language.png', dpi=300, bbox_inches='tight')
    plt.close()

# Call the function to test
plot_metrics_by_language(lang_results)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

languages = list(lang_results.keys())

# Assuming you have a dictionary of languages and their error scores
language_errors = {lang: lang_results[lang]['mae'] for lang in languages if lang != 'esp'}

print(language_errors)

# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(language_errors, orient='index', columns=['error_score'])

# Calculate the correlation matrix
correlation_matrix = df['error_score'].apply(lambda x: np.abs(x - df['error_score']))

# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='YlGnBu', fmt='.2f', square=True)

# Set the title
plt.title('Language Error Score Correlation Heatmap')

# Show the plot
plt.show()

In [ ]:
def plot_error_distributions(model, dataloader, device, languages):
    model.eval()
    errors = {lang: [] for lang in languages}

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            batch_languages = batch['language']

            outputs = model(input_ids, attention_mask)
            predictions = outputs.squeeze().cpu().numpy()
            true_labels = labels.cpu().numpy()

            batch_errors = predictions - true_labels

            for error, lang in zip(batch_errors, batch_languages):
                errors[lang].append(error)

    fig, axes = plt.subplots(len(languages), 1, figsize=(12, 4*len(languages)))
    fig.suptitle('Error Distributions by Language', fontsize=16)

    for i, lang in enumerate(languages):
        sns.histplot(errors[lang], kde=True, ax=axes[i])
        axes[i].set_title(f'{lang} Error Distribution')
        axes[i].set_xlabel('Error')
        axes[i].set_ylabel('Count')

    plt.tight_layout()
    plt.savefig('error_distributions.png', dpi=300, bbox_inches='tight')
    plt.close()

# Call the function after evaluation
plot_error_distributions(model, test_dataloader, device, list(lang_results.keys()))
